# 어텐션 메커니즘과 트랜스포머

순환 신경망(Recurrent Neural Networks, RNN)의 주요 단점 중 하나는 시퀀스 내 모든 단어가 결과에 동일한 영향을 미친다는 점입니다. 이는 이름 엔티티 인식(Named Entity Recognition)이나 기계 번역(Machine Translation)과 같은 시퀀스-투-시퀀스(Sequence-to-Sequence) 작업에서 표준 LSTM 인코더-디코더 모델의 성능을 저하시키는 원인이 됩니다. 실제로 입력 시퀀스의 특정 단어는 다른 단어들보다 순차적 출력에 더 큰 영향을 미치는 경우가 많습니다.

기계 번역과 같은 시퀀스-투-시퀀스 모델을 생각해봅시다. 이 모델은 두 개의 순환 신경망으로 구현되며, 하나의 네트워크(**인코더**)는 입력 시퀀스를 은닉 상태로 압축하고, 다른 네트워크(**디코더**)는 이 은닉 상태를 번역된 결과로 펼칩니다. 이 접근 방식의 문제는 네트워크의 최종 상태가 문장의 시작 부분을 기억하기 어렵다는 점이며, 이는 긴 문장에서 모델의 품질 저하를 초래합니다.

**어텐션 메커니즘**은 RNN의 각 출력 예측에 대해 각 입력 벡터의 맥락적 영향을 가중치로 부여하는 방법을 제공합니다. 이는 입력 RNN의 중간 상태와 출력 RNN 사이에 지름길을 생성함으로써 구현됩니다. 이 방식으로 출력 심볼 $y_t$를 생성할 때, 서로 다른 가중치 계수 $\alpha_{t,i}$를 사용하여 모든 입력 은닉 상태 $h_i$를 고려합니다.

![어텐션 레이어가 추가된 인코더/디코더 모델을 보여주는 이미지](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)의 어텐션 메커니즘이 포함된 인코더-디코더 모델, [이 블로그 글](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)에서 인용됨*

어텐션 행렬 $\{\alpha_{i,j}\}$은 출력 시퀀스의 특정 단어를 생성하는 데 있어 특정 입력 단어가 얼마나 중요한지를 나타냅니다. 아래는 이러한 행렬의 예시입니다:

![RNNsearch-50에 의해 발견된 샘플 정렬을 보여주는 이미지, Bahdanau - arviz.org에서 발췌](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)에서 발췌한 그림 (Fig.3)*

어텐션 메커니즘은 현재 또는 근접한 자연어 처리(NLP) 분야의 최첨단 기술에 큰 기여를 하고 있습니다. 하지만 어텐션을 추가하면 모델 파라미터 수가 크게 증가하여 RNN에서 확장성 문제가 발생합니다. RNN의 확장성을 제한하는 주요 제약은 모델의 순환적 특성으로 인해 학습을 배치 처리하거나 병렬화하기 어렵다는 점입니다. RNN에서는 시퀀스의 각 요소를 순차적으로 처리해야 하므로 병렬화가 쉽지 않습니다.

어텐션 메커니즘의 도입과 이러한 제약 조건은 오늘날 우리가 알고 사용하는 BERT에서 OpenGPT3에 이르는 최첨단 트랜스포머 모델의 탄생으로 이어졌습니다.

## 트랜스포머 모델

이전 예측의 맥락을 다음 평가 단계로 전달하는 대신, **트랜스포머 모델**은 **위치 인코딩(Positional Encoding)**과 **어텐션**을 사용하여 주어진 텍스트 창 내에서 입력의 맥락을 캡처합니다. 아래 이미지는 위치 인코딩과 어텐션이 주어진 창 내에서 맥락을 어떻게 캡처하는지 보여줍니다.

![트랜스포머 모델에서 평가가 수행되는 방식을 보여주는 애니메이션 GIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

각 입력 위치가 독립적으로 각 출력 위치에 매핑되기 때문에, 트랜스포머는 RNN보다 병렬화가 더 잘 이루어질 수 있으며, 이는 훨씬 더 크고 표현력이 뛰어난 언어 모델을 가능하게 합니다. 각 어텐션 헤드는 단어 간의 다양한 관계를 학습하는 데 사용될 수 있으며, 이는 자연어 처리 작업의 성능을 향상시킵니다.

## 간단한 트랜스포머 모델 구축

Keras에는 내장된 트랜스포머 레이어가 없지만, 직접 구축할 수 있습니다. 이전과 마찬가지로 AG News 데이터셋의 텍스트 분류에 초점을 맞추겠지만, 트랜스포머 모델은 더 어려운 NLP 작업에서 최고의 결과를 보여준다는 점을 언급할 가치가 있습니다.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Keras의 새로운 레이어는 `Layer` 클래스를 서브클래스로 만들어야 하며, `call` 메서드를 구현해야 합니다. **Positional Embedding** 레이어부터 시작해 보겠습니다. 우리는 [공식 Keras 문서의 일부 코드](https://keras.io/examples/nlp/text_classification_with_transformer/)를 사용할 것입니다. 모든 입력 시퀀스를 길이 `maxlen`으로 패딩한다고 가정하겠습니다.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

이 레이어는 두 개의 `Embedding` 레이어로 구성됩니다: 하나는 토큰을 임베딩하기 위한 것이고(앞서 논의한 방식으로), 다른 하나는 토큰 위치를 임베딩하기 위한 것입니다. 토큰 위치는 `tf.range`를 사용하여 0부터 `maxlen`까지의 자연수 시퀀스로 생성된 후, 임베딩 레이어를 거칩니다. 이렇게 생성된 두 개의 임베딩 벡터는 더해져서 입력의 위치 정보를 포함한 임베딩 표현을 생성하며, 이 표현의 형태는 `maxlen`$\times$`embed_dim`입니다.

이제, 트랜스포머 블록을 구현해 보겠습니다. 이는 이전에 정의한 임베딩 레이어의 출력을 입력으로 받게 됩니다:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

이제 완전한 Transformer 모델을 정의할 준비가 되었습니다:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Models

**BERT**(Bidirectional Encoder Representations from Transformers)는 매우 큰 다층 트랜스포머 네트워크로, *BERT-base*는 12개의 레이어, *BERT-large*는 24개의 레이어로 구성됩니다. 이 모델은 먼저 대규모 텍스트 데이터(WikiPedia + 책)를 사용하여 비지도 학습(문장에서 마스킹된 단어를 예측)을 통해 사전 학습됩니다. 사전 학습 과정에서 모델은 상당한 수준의 언어 이해를 습득하며, 이를 다른 데이터셋에 미세 조정을 통해 활용할 수 있습니다. 이 과정을 **전이 학습**이라고 합니다.

![picture from http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

BERT, DistilBERT, BigBird, OpenGPT3 등 다양한 트랜스포머 아키텍처 변형이 있으며, 이들 모두 미세 조정이 가능합니다.

이제 사전 학습된 BERT 모델을 사용하여 기존의 시퀀스 분류 문제를 해결하는 방법을 살펴보겠습니다. [공식 문서](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)에서 아이디어와 일부 코드를 가져와 사용해 보겠습니다.

사전 학습된 모델을 로드하기 위해 **Tensorflow hub**를 사용할 것입니다. 먼저, BERT 전용 벡터라이저를 로드해 보겠습니다:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

원래 네트워크가 학습된 벡터라이저와 동일한 벡터라이저를 사용하는 것이 중요합니다. 또한, BERT 벡터라이저는 세 가지 구성 요소를 반환합니다:
* `input_word_ids`: 입력 문장의 토큰 번호로 이루어진 시퀀스
* `input_mask`: 시퀀스에서 실제 입력이 포함된 부분과 패딩 부분을 나타냅니다. 이는 `Masking` 레이어가 생성하는 마스크와 유사합니다.
* `input_type_ids`: 언어 모델링 작업에 사용되며, 하나의 시퀀스에서 두 개의 입력 문장을 지정할 수 있도록 합니다.

그런 다음, BERT 특징 추출기를 인스턴스화할 수 있습니다:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


그래서 BERT 레이어는 여러 유용한 결과를 반환합니다:
* `pooled_output`은 시퀀스 내 모든 토큰을 평균화한 결과입니다. 이를 네트워크 전체의 지능적인 의미적 임베딩으로 볼 수 있습니다. 이는 이전 모델에서 사용한 `GlobalAveragePooling1D` 레이어의 출력과 동일합니다.
* `sequence_output`은 마지막 Transformer 레이어의 출력입니다 (위 모델에서 `TransformerBlock`의 출력에 해당).
* `encoder_outputs`는 모든 Transformer 레이어의 출력입니다. 우리가 4-레이어 BERT 모델을 로드했기 때문에 (이름에 `4_H`가 포함되어 있다는 점에서 추측할 수 있듯이), 4개의 텐서를 포함합니다. 마지막 텐서는 `sequence_output`과 동일합니다.

이제 엔드-투-엔드 분류 모델을 정의하겠습니다. 우리는 *함수형 모델 정의*를 사용할 것입니다. 모델 입력을 정의한 후, 일련의 표현식을 통해 출력 값을 계산합니다. 또한 BERT 모델의 가중치를 학습하지 않도록 설정하고, 최종 분류기만 학습시킬 것입니다:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


훈련 가능한 매개변수가 적음에도 불구하고, BERT 특징 추출기가 계산적으로 무겁기 때문에 과정이 꽤 느립니다. 훈련 부족이나 모델 매개변수 부족으로 인해 합리적인 정확도를 달성하지 못한 것 같습니다.

이제 BERT 가중치를 고정 해제하고 함께 훈련해 봅시다. 이를 위해서는 매우 작은 학습률이 필요하며, **warmup**과 **AdamW** 옵티마이저를 사용하는 더 신중한 훈련 전략이 요구됩니다. 옵티마이저를 생성하기 위해 `tf-models-official` 패키지를 사용할 것입니다.


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

훈련이 꽤 느리게 진행되는 것을 볼 수 있습니다. 하지만 몇 번의 에포크(5-10) 동안 모델을 훈련시키고, 이전에 사용했던 접근 방식과 비교하여 최상의 결과를 얻을 수 있는지 실험해보는 것도 좋습니다.

## Huggingface Transformers 라이브러리

Transformer 모델을 사용하는 또 다른 매우 일반적이고 (조금 더 간단한) 방법은 [HuggingFace 패키지](https://github.com/huggingface/)를 사용하는 것입니다. 이 패키지는 다양한 NLP 작업을 위한 간단한 빌딩 블록을 제공합니다. Tensorflow와 PyTorch(또 다른 매우 인기 있는 신경망 프레임워크) 모두에서 사용할 수 있습니다.

> **Note**: Transformers 라이브러리가 어떻게 작동하는지에 관심이 없다면 이 노트북의 끝부분으로 건너뛰어도 됩니다. 위에서 했던 것과 본질적으로 크게 다르지 않은 내용을 보게 될 것입니다. 우리는 다른 라이브러리와 훨씬 더 큰 모델을 사용하여 BERT 모델을 훈련시키는 동일한 단계를 반복할 것입니다. 따라서 이 과정은 다소 긴 훈련을 포함하므로, 코드를 훑어보는 것으로 충분할 수도 있습니다.

[Huggingface Transformers](http://huggingface.co)를 사용하여 우리의 문제를 어떻게 해결할 수 있는지 살펴봅시다.


먼저 사용할 모델을 선택해야 합니다. Huggingface에는 몇 가지 내장 모델 외에도 [온라인 모델 저장소](https://huggingface.co/models)가 있어, 커뮤니티에서 제공하는 더 많은 사전 학습된 모델을 찾을 수 있습니다. 이러한 모든 모델은 모델 이름만 제공하면 로드하고 사용할 수 있습니다. 모델에 필요한 모든 바이너리 파일은 자동으로 다운로드됩니다.

때로는 직접 만든 모델을 로드해야 할 수도 있습니다. 이 경우, 토크나이저의 매개변수, 모델 매개변수가 포함된 `config.json` 파일, 바이너리 가중치 등 관련 파일이 포함된 디렉터리를 지정할 수 있습니다.

모델 이름을 통해 모델과 토크나이저를 모두 인스턴스화할 수 있습니다. 토크나이저부터 시작해 봅시다:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` 객체는 텍스트를 직접 인코딩할 수 있는 `encode` 함수를 포함하고 있습니다:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

우리는 토크나이저를 사용하여 시퀀스를 모델에 전달하기 적합한 방식으로 인코딩할 수도 있습니다. 즉, `token_ids`, `input_mask` 필드 등을 포함하는 방식입니다. 또한 `return_tensors='tf'` 인수를 제공하여 Tensorflow 텐서를 원한다고 지정할 수도 있습니다.


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

우리의 경우, 우리는 `bert-base-uncased`라는 사전 학습된 BERT 모델을 사용할 것입니다. *Uncased*는 모델이 대소문자를 구분하지 않는다는 것을 의미합니다.

모델을 훈련할 때, 우리는 토큰화된 시퀀스를 입력으로 제공해야 하므로 데이터 처리 파이프라인을 설계할 것입니다. `tokenizer.encode`는 Python 함수이기 때문에, 이전 단원에서 사용했던 것처럼 `py_function`을 호출하여 동일한 접근 방식을 사용할 것입니다.


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

이제 `BertForSequenceClassification` 패키지를 사용하여 실제 모델을 로드할 수 있습니다. 이를 통해 모델이 분류를 위한 필요한 아키텍처를 이미 갖추고 있으며, 최종 분류기를 포함하고 있음을 보장합니다. 최종 분류기의 가중치가 초기화되지 않았으며 모델이 사전 훈련이 필요하다는 경고 메시지가 표시될 것입니다. 이는 완전히 괜찮습니다. 왜냐하면 바로 그것이 우리가 하려는 일이기 때문입니다!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


`summary()`에서 볼 수 있듯이, 모델은 거의 1억 1천만 개의 파라미터를 포함하고 있습니다! 아마도 비교적 작은 데이터셋에서 간단한 분류 작업을 원한다면, BERT 기본 레이어를 훈련시키고 싶지 않을 것입니다:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


이제 훈련을 시작할 준비가 되었습니다!

> **Note**: 전체 규모의 BERT 모델을 훈련하는 데는 매우 많은 시간이 소요될 수 있습니다! 따라서 우리는 처음 32개의 배치만 훈련할 것입니다. 이는 모델 훈련이 어떻게 설정되는지 보여주기 위한 것입니다. 전체 규모의 훈련을 시도해보고 싶다면 `steps_per_epoch`와 `validation_steps` 매개변수를 제거하고 기다릴 준비를 하세요!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


만약 반복 횟수를 늘리고 충분히 기다리며 여러 에포크 동안 훈련을 진행한다면, BERT 분류가 최고의 정확도를 제공할 것으로 기대할 수 있습니다! 이는 BERT가 이미 언어 구조를 꽤 잘 이해하고 있기 때문에, 최종 분류기를 미세 조정(fine-tuning)하는 것만 필요하기 때문입니다. 하지만, BERT는 대규모 모델이기 때문에 전체 훈련 과정이 오래 걸리고 상당한 계산 능력이 필요합니다! (GPU, 그리고 가능하다면 여러 개의 GPU가 필요합니다).

> **Note:** 이 예제에서는 가장 작은 사전 훈련된 BERT 모델 중 하나를 사용했습니다. 더 큰 모델들은 더 나은 결과를 제공할 가능성이 높습니다.


## 주요 내용

이번 단원에서는 **transformers**를 기반으로 한 최신 모델 아키텍처를 살펴보았습니다. 이를 텍스트 분류 작업에 적용해 보았지만, 마찬가지로 BERT 모델은 개체 추출, 질문 응답, 기타 NLP 작업에도 사용할 수 있습니다.

Transformer 모델은 현재 NLP 분야에서 최첨단 기술을 대표하며, 대부분의 경우 맞춤형 NLP 솔루션을 구현할 때 가장 먼저 실험해봐야 할 솔루션입니다. 하지만, 이 모듈에서 다룬 순환 신경망의 기본 원리를 이해하는 것은 고급 신경망 모델을 구축하려면 매우 중요합니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
